Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from azure.core.credentials import AzureKeyCredential
from msrest.authentication import CognitiveServicesCredentials
import cv2
import pytesseract
import numpy as np
from jiwer import wer, cer
from array import array
from Levenshtein import distance as levenshtein_distance, hamming
from dotenv import dotenv_values
import time

In [14]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''
config = dotenv_values(".env")
subscription_key = config["VISION_KEY"]
endpoint = config["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [15]:
import cv2
import numpy as np


# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


In [16]:
image_path = "test1.png"
image = cv2.imread(image_path)
image_gray = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
image_thresh = thresholding(image_gray)


In [27]:

def ocr_tesseract(image):
    return pytesseract.image_to_string(image, lang='eng')

def ocr_azure(image_path):

    with open(image_path, 'rb') as image_file:
        read_response = computervision_client.read_in_stream(
        image=image_file,
        mode="Printed",
        raw=True
        )

        operation_id = read_response.headers['Operation-Location'].split('/')[-1]
        while True:
            read_result = computervision_client.get_read_result(operation_id)
            if read_result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)

        result = []
        if read_result.status == OperationStatusCodes.succeeded:
            for text_result in read_result.analyze_result.read_results:
                for line in text_result.lines:
                    print(line.text)
                    result.append(line.text)
    
    return " ".join([line for line in result])

def evaluate_recognition(ground_truth, recognized_text):
    lev_dist = levenshtein_distance(ground_truth, recognized_text)
    word_err = wer(ground_truth, recognized_text)
    char_err = cer(ground_truth, recognized_text)

    # Hamming distance (only valid for strings of the same length)
    if len(ground_truth) == len(recognized_text):
        hamming_dist = hamming(ground_truth, recognized_text)
    else:
        hamming_dist = None  # Hamming distance is undefined for different lengths



    return {
        "Levenshtein Distance": lev_dist,
        "Word Error Rate": word_err,
        "Character Error Rate": char_err,
        "Hamming Distance": hamming_dist,
    }


In [28]:




ground_truth_text = ["Google Cloud Platform"]

# OCR using Tesseract
tesseract_result = ocr_tesseract(image_thresh)
print("Tesseract OCR Result:\n", tesseract_result)

# OCR using Azure (replace with your credentials)
azure_result = ocr_azure(image_path)
print("Azure OCR Result:\n", azure_result)

# Evaluation
print("Tesseract Evaluation:", evaluate_recognition(ground_truth_text, tesseract_result))
print("Azure Evaluation:", evaluate_recognition(ground_truth_text, azure_result))


Tesseract OCR Result:
 
Google Cloud
Platform
Azure OCR Result:
 Google Cloud Platform
Tesseract Evaluation: {'Levenshtein Distance': 1, 'Word Error Rate': 1.0, 'Character Error Rate': 1.0, 'Hamming Distance': None}
Azure Evaluation: {'Levenshtein Distance': 21, 'Word Error Rate': 0.0, 'Character Error Rate': 0.0, 'Hamming Distance': None}


In [ ]:
h, w,c = image.shape
boxes = pytesseract.image_to_boxes(image_thresh)

# Draw rectangles around detected text
for b in boxes.splitlines():
    b = b.split(' ')
    x1, y1, x2, y2 = int(b[1]), int(b[2]), int(b[3]), int(b[4])

    # Fix y-coordinates by subtracting from image height
    y1, y2 = h - y1, h - y2

    # Draw the bounding box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Display the image
cv2.imshow('img', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


: 

In [ ]:
# Perform OCR
with open(image_path, 'rb') as image_file:
        ocr_result = computervision_client.read_in_stream(
        image=image_file,mode="Printed",raw=True)

        operation_location = ocr_result.headers["Operation-Location"]
        operation_id = operation_location.split("/")[-1]


        while True:
            result = computervision_client.get_read_result(operation_id)
            if result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)

        if result.status == OperationStatusCodes.succeeded:
            read_results = result.analyze_result.read_results
            h, w, c = image.shape
            for read_result in read_results:
                for line in read_result.lines:
                    for word in line.words:
                        bbox = word.bounding_box
                        x1, y1 = int(bbox[0]), int(bbox[1])
                        x2, y2 = int(bbox[4]), int(bbox[5])
                        image_color = cv2.rectangle(image_color, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Display the image
        cv2.imshow('img', image_color)
        cv2.waitKey(0)
        cv2.destroyAllWindows()